## Import packages

In [1]:
import argparse
import numpy as np
import time
from collections.abc import Iterable
import robosuite
import robosuite.utils.transform_utils as T
from robosuite.wrappers import IKWrapper
import matplotlib.pyplot as plt
from robosuite.utils.mjcf_utils import array_to_string, string_to_array
from motion_planner import approach_to_object, move_to_6Dpos, get_camera_pos, force_gripper, move_to_pos_lrarm, move_to_pos, get_pos_and_rot

## Function for getting a vertical image
### arena_pos: pose of a bin
### vis_on (default False): whether plotting the image or not

In [2]:
def get_vertical_image(env, arena_pos, vis_on=False):
    camera_id = env.sim.model.camera_name2id("eye_on_wrist")
    camera_obs = env.sim.render(
        camera_name=env.camera_name,
        width=env.camera_width,
        height=env.camera_height,
        depth=env.camera_depth
    )
    pnt_hem, rot_mat = get_camera_pos(arena_pos, np.array([0.0, 0.0, 0.0]))
    env.sim.data.cam_xpos[camera_id] = pnt_hem
    env.sim.data.cam_xmat[camera_id] = rot_mat.flatten()

    camera_obs = env.sim.render(
        camera_name=env.camera_name,
        width=env.camera_width,
        height=env.camera_height,
        depth=env.camera_depth
    )
    if env.camera_depth:
        vertical_color_image, ddd = camera_obs

    extent = env.mjpy_model.stat.extent
    near = env.mjpy_model.vis.map.znear * extent
    far = env.mjpy_model.vis.map.zfar * extent

    vertical_depth_image = near / (1 - ddd * (1 - near / far))
    vertical_depth_image = np.where(vertical_depth_image > 0.25, vertical_depth_image, 1)

    if vis_on:
        plt.imshow(np.flip(vertical_color_image, axis=0))
        plt.show()

        plt.imshow(np.flip(vertical_depth_image, axis=0), cmap='gray')
        plt.show()

    return np.flip(vertical_color_image, axis=0), np.flip(vertical_depth_image, axis=0)

## Drop objects in the bin sequentially
### To ensure all objects placed in the bin

In [3]:
def random_quat(rand=None):
    if rand is None:
        rand = np.random.rand(3)
    else:
        assert len(rand) == 3
    r1 = np.sqrt(1.0 - rand[0])
    r2 = np.sqrt(rand[0])
    pi2 = np.pi * 2.0
    t1 = pi2 * rand[1]
    t2 = pi2 * rand[2]
    return np.array(
        (np.sin(t1) * r1, np.cos(t1) * r1, np.sin(t2) * r2, np.cos(t2) * r2),
        dtype=np.float32,
    )    

def place_objects_in_bin(env, num_objects=10):
    for i in range(num_objects):
        object_z = env.env.model.bin_size[2] + 0.5
        object_x = 0.0
        object_y = 0.0
        object_xyz = np.array([object_x, object_y, object_z])
        pos = env.env.model.bin_offset + object_xyz
        env.env.model.objects[i].set("pos", array_to_string(pos))
        quat = random_quat()
        env.env.model.objects[i].set("quat", array_to_string(quat))
        env.reset_sims()
        for j in range(0, 500):
            env.env.sim.step()
        for k in range(0, i + 1):
            obj_str = env.env.item_names[k]
            obj_id = env.env.obj_body_id[obj_str]
            pos = env.env.sim.data.body_xpos[obj_id]
            quat = env.env.sim.data.body_xquat[obj_id]
            env.env.model.objects[k].set("pos", array_to_string(pos))
            env.env.model.objects[k].set("quat", array_to_string(quat))
    env.reset_sims()

    bin_pos = env.env.model.bin_offset
    for i in range(num_objects):
        obj_str = env.env.item_names[i]
        obj_id = env.env.obj_body_id[obj_str]
        pos = env.env.sim.data.body_xpos[obj_id]
        if abs(pos[0] - bin_pos[0]) < env.bin_size[0] / 2.0 and abs(pos[1] - bin_pos[1]) < env.bin_size[1] / 2.0:
            continue
        else:
            object_z = env.env.model.bin_size[2] + 0.3
            object_x = 0.1 * np.random.random() - 0.05
            object_y = 0.1 * np.random.random() - 0.05
            object_xyz = np.array([object_x, object_y, object_z])
            pos = bin_pos + object_xyz
            env.env.model.objects[i].set("pos", array_to_string(pos))
            env.reset_sims()
            for j in range(0, 500):
                env.env.sim.step()
            for k in range(0, num_objects):
                obj_str = env.env.item_names[k]
                obj_id = env.env.obj_body_id[obj_str]
                pos = env.env.sim.data.body_xpos[obj_id]
                quat = env.env.sim.data.body_xquat[obj_id]
                env.env.model.objects[k].set("pos", array_to_string(pos))
                env.env.model.objects[k].set("quat", array_to_string(quat))
        env.reset_sims()

    item_list = []
    pos_list = []
    quat_list = []
    for i in range(num_objects):
        obj_str = env.item_names[i]
        obj_id = env.obj_body_id[obj_str]
        pos = env.sim.data.body_xpos[obj_id]
        quat = env.sim.data.body_xquat[obj_id]
        item_list.append(obj_str.split("_")[1])
        pos_list.append(pos)
        quat_list.append(quat)
    return item_list, pos_list, quat_list

## Set parameters for the simulation
### seed: random seed
### num_objects: the number of graspable objects
### num_episodes: the number of simulation episodes
### num_steps: the number of steps for each epsiode
### render : display all grasping procedures if it is true
### bin_type: table or bin
### object_type: T, L, or 3DNet meshes

In [4]:
seed = 0
num_objects = 5
num_episodes = 100
num_steps = 5
render = True
bin_type = "table"
object_type = "T"

## Load MuJoCo baxter environements

In [5]:
np.random.seed(seed)
env = robosuite.make(
    "BaxterSteepedBinCollectData",
    bin_type=bin_type,
    object_type=object_type,
    ignore_done=True,
    has_renderer=True,
    camera_name="eye_on_wrist",
    gripper_visualization=True,
    use_camera_obs=False,
    camera_depth=True,
    num_objects=num_objects,
    control_freq=100
)
env = IKWrapper(env)

Creating window glfw
FCGQCNNModelFactory INFO     Initializing FC-GQ-CNN with Tensorflow as backend...
FCGQCNNTF  INFO     Building Network...
FCGQCNNTF  INFO     Building Image Stream...
FCGQCNNTF  INFO     Building convolutional layer: conv1_1...
FCGQCNNTF  INFO     Building convolutional layer: conv1_2...
FCGQCNNTF  INFO     Building convolutional layer: conv2_1...
FCGQCNNTF  INFO     Building convolutional layer: conv2_2...
FCGQCNNTF  INFO     Converting fc layer fc3 to fully convolutional...
FCGQCNNTF  INFO     Converting fc layer fc4 to fully convolutional...
FCGQCNNTF  INFO     Converting fc layer fc5 to fully convolutional...
FCGQCNNTF  INFO     Building Pair-wise Softmax Layer...
FCGQCNNTF  INFO     Initializing TF Session...


## cam_offset: differences of x, y and z coordinates between the end effector of Baxter and the camera
## camera_id: to set camera pose as we want, we need to know id of the camera on the robot arm 
## arena_pos: pose of the bin containing objects
## init_pos: initial pose of the end effector
## release_pos: where to release objects after the success of grasps

In [1]:
cam_offset = np.array([0.05, 0, 0.15855])
camera_id = env.sim.model.camera_name2id("eye_on_wrist")

arena_pos = env.env.mujoco_arena.bin_abs
init_pos = arena_pos + np.array([0, 0, 0.7]) - cam_offset
release_pos = arena_pos + np.array([0, 0.59, 0.3])

NameError: name 'np' is not defined

## Collect grasping dataset
### Move to the inital position
### Sample one approaching vector
### Get the image viewed at the approaching vector and find the best grasp using the pretrained gqcnn
### Move to target point with hover distance
### Move to real target position
### Try the grasp
### Check collision
#### If success, go back to the first step
#### If failure, move the robot arm to the release pos and release the holding object and go back to the first step

In [1]:
for i in range(0, num_episodes):
    print("Reset!")
    item_list, pos_list, quat_list = place_objects_in_bin(env, num_objects=env.num_objects)
    print("Place all objects in bin!")
    vt_c_im_list, vt_d_im_list, rot_c_im_list, rot_d_im_list = [], [], [], []
    vt_g_pos_list, vt_g_euler_list, rot_g_pos_list, rot_g_euler_list = [], [], [], []
    g_label_list = []
    success_count, failure_count, controller_failure = 0, 0, 0

    for step in range(num_steps):
        ### Move to the inital position
        stucked = move_to_pos_lrarm(env, np.array([0.4, 0.6, 1.0]), np.array([0.4, -0.6, 1.0]), level=1.0, render=render)
        if stucked == -1:
            controller_failure += 1
            continue

        ### Sample one approaching vector
        vt_c_im, vt_d_im = get_vertical_image(env, arena_pos, vis_on=False)
        phi = np.pi / 6.0 * np.random.random()
        theta = 2 * np.pi * np.random.random()

        camera_obs = env.sim.render(
            camera_name=env.camera_name,
            width=env.camera_width,
            height=env.camera_height,
            depth=env.camera_depth
        )
        pnt_hem, rot_mat = get_camera_pos(arena_pos, np.array([0, phi, theta]))
        env.sim.data.cam_xpos[camera_id] = pnt_hem
        env.sim.data.cam_xmat[camera_id] = rot_mat.flatten()

        sel_start = time.time()

        ### Run FCGQQCNN
        print("Try FC-GQ-CNN")
        [result, depths, d_im], rot_c_im, rot_d_im = env.env.gqcnn(vis_on=False, num_candidates=10) ## vis_on 

        if isinstance(result, Iterable):
            sample_grasp_idx = np.random.randint(10, size=1)
            result = result[sample_grasp_idx[0]].grasp
        else:
            if result is None:
                continue
            else:
                result = result.grasp

        p_x, p_y = result.center
        print('p_x, p_y: ', p_x, p_y)

        graspZ = result.depth
        print('graspZ: ', graspZ)

        dx, dy = env.env._pixel2pos(p_x, p_y, graspZ, arena_pos=[0, 0, 0])
        dz = graspZ

        # Move to target point with hover distance
        t_pos = pnt_hem
        t_angle = [result.angle, phi, theta]

        stucked = move_to_6Dpos(env, t_pos, t_angle, cam_offset=[-dx, dy, dz - 0.10], grasp=0.0, level=4.0, render=render)
        if stucked == -1:
            controller_failure += 1
            continue
        print('arrived at the hover point')

        vt_c_im_list.append(vt_c_im)
        vt_d_im_list.append(vt_d_im)
        rot_c_im_list.append(rot_c_im)
        rot_d_im_list.append(rot_d_im)
        vt_g_pos_list.append(t_pos)
        vt_g_euler_list.append(t_angle)
        rot_g_pos_list.append([dx, dy, dz])
        rot_g_euler_list.append([result.angle, 0, 0])

        # Move to real target position
        stucked = approach_to_object(env, t_pos, t_angle, cam_offset=[-dx, dy, dz + 0.01], grasp=0.0, level=2.0, render=render)
        if stucked == -1:
            failure_count += 1
            g_label_list.append(0.0)
            print('Grasping Failed, Success rate: ', success_count / (success_count + failure_count + controller_failure), success_count / (success_count + failure_count), 
                ', total trial: ', success_count, failure_count, controller_failure)
            stucked = move_to_6Dpos(env, t_pos, t_angle, cam_offset=[-dx, dy, dz - 0.10], grasp=0.0, level=1.0, render=render)
            continue

        # Grasping
        obs = env.env._get_observation()
        print('arrived at:', obs['right_eef_pos'], '(before grasping)')

        stucked = force_gripper(env, grasp=1.0, render=render)
        print("Try grasping!")
        stucked = move_to_6Dpos(env, t_pos, t_angle, cam_offset=[-dx, dy, dz - 0.30], grasp=1.0, level=0.1, render=render)
        print("Go back to the hover point.")

        # Check collision - success/failure
        collision = env._check_contact()
        if collision:
            success_count += 1
            g_label_list.append(1.0)
            print('Grasping Success, Success rate: ', success_count / (success_count + failure_count + controller_failure), success_count / (success_count + failure_count), 
                ', total trial: ', success_count, failure_count, controller_failure)
            move_to_pos(env, release_pos, 0.0, 1.0, level=0.1, render=render)
            force_gripper(env, grasp=0.0, render=render)
        else:
            failure_count += 1
            g_label_list.append(0.0)
            print('Grasping Failed, Success rate: ', success_count/ (success_count + failure_count + controller_failure), success_count / (success_count + failure_count), 
                ', total trial: ', success_count, failure_count, controller_failure)

    env.env.reset_objects()
    save_file_name = "data/6DoF_grasp/episode_" + str(i) + ".npz"
    np.savez(save_file_name, item_list, pos_list, quat_list, vt_c_im_list, vt_d_im_list, rot_c_im_list, rot_d_im_list, 
        vt_g_pos_list, vt_g_euler_list, rot_g_pos_list, rot_g_euler_list, g_label_list)

NameError: name 'num_episodes' is not defined